# TP diffusion (réseau débruiteur)
Ce TP est une implémentation du modèle génératif profond ["Denoising Diffusion Probabilistic Models"](https://arxiv.org/abs/2006.11239) étudié en cours. Comme lors du TP VAE, le problème considéré est en 1D afin de pouvoir réaliser des affichages d'une part et de réduire le plus possible le temps d'entraînement d'autre part. **L'implémentation se fera en PyTorch.**
1. Lancer une session linux (et non pas windows)
2. Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
3. Dans ce terminal, taper la commande suivante pour lancer Spyder : `spyder &`
4. Configurer Spyder en suivant ces instructions : [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
5. Créer un dossier `TP_diffusion`.
6. Créer un script python `tp.py` dans le dossier `TP_diffusion` et coller les lignes de code suivantes : 

In [1]:
import math
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import sys

# Mélange de gaussiennes 1D
Afin de s'assurer du bon fonctionnement de l'approche, nous allons considérer un cas d'école où $p_{data}(x)$ est **connue** et facile à échantillonner. Nous considérerons que $p_{data}(x)$ est un mélange de gaussiennes, ainsi la diffusion aura également une forme analytique (car "diffuser" un mélange de gaussiennes revient à "diffuser" chaque gaussienne du mélange). **Rappelons que dans un cas réel, $p_{data}(x)$ n'est pas connue. Ici, le fait de connaître $p_{data}(x)$ nous permet de faire de faire plus d'affichages pour mieux comprendre ce qu'il se passe.**  
  
Copier/coller les trois fonctions suivantes permettant d'afficher $p_{data}(x)$ et sa diffusion, d'afficher une simple gaussienne centrée réduite (ce qui nous permettra de vérifier visuellement qu'à la fin de la diffusion la distribution est très proche d'une gaussienne centrée réduite), et d'échantillonner $p_{data}(x)$.

In [ ]:
def plot_pdata_diffusion(w_list, mu_list, sigma_list, alpha_diff, ax=None, orientation='vertical'):
    
    x = torch.linspace(start=-6., end=8., steps=1000)
    
    p_data = torch.zeros_like(x)
    N_comp = w_list.shape[0]
    for i in range(N_comp):
        std_diff = math.sqrt(alpha_diff*(sigma_list[i]**2)+(1.-alpha_diff))
        p_data_i = (1./(std_diff*math.sqrt(2*math.pi))*torch.exp(-0.5*((x-np.sqrt(alpha_diff)*mu_list[i])/std_diff)**2))
        p_data += w_list[i]*p_data_i
        
    if(ax==None):
        if(orientation=='vertical'):
            plt.plot(x,p_data,'k')
        else:
            plt.plot(p_data,x,'k')
    else:
        if(orientation=='vertical'):
            ax.plot(x,p_data,'k')
        else:
            ax.plot(p_data,x,'k')
    return

def plot_gaussian(ax=None, orientation='vertical'):
    
    x = torch.linspace(start=-6., end=8., steps=1000)
        
    p_data = (1./(math.sqrt(2*math.pi))*torch.exp(-0.5*(x)**2))
        
    if(ax==None):
        if(orientation=='vertical'):
            plt.plot(x,p_data,'r-')
        else:
            plt.plot(p_data,x,'r-')
    else:
        if(orientation=='vertical'):
            ax.plot(x,p_data,'r-')
        else:
            ax.plot(p_data,x,'r-')
    return

def sample_from_pdata(N, w_list, mu_list, sigma_list):
    
    n_c = w_list.shape[0]

    samp = torch.zeros((N,1))
    mask = torch.multinomial(w_list,num_samples=N,replacement=True)
    
    for i in range(n_c):
        samp_i = torch.normal(mean=mu_list[i], std=sigma_list[i], size=(N,1))
        samp[mask==i] = samp_i[mask==i]
 
    return samp

Définir les paramètres du mélange de gaussiennes et afficher $p_{data}(x)$ :

In [ ]:
w_list = torch.tensor([0.2, 0.45, 0.35, 0.1, 0.3]) #poids du mélange de gaussiennes
w_list /=w_list.sum() 

mu_list = torch.tensor([-3., 2.5, 1.5, -2.5, 5.]) #moyenne de chaque composante
sigma_list = torch.tensor([0.3, 1.2, 0.3, 0.2, 0.1]) #écart-type de chaque composante

plt.figure(1)
plot_pdata_diffusion(w_list, mu_list, sigma_list, 1)

Nous pouvons désormais générer notre base de données en tirant des échantillons selon $p_{data}(x)$ (**RAPPEL : Dans un cas réel, ces échantillons sont donnés et $p_{data}(x)$ est inconnue. L'objectif d'un modèle génératif est justement d'apprendre à générer de nouveaux échantillons !**) :

In [ ]:
N_samp = int(2e4)
X = sample_from_pdata(N_samp, w_list, mu_list, sigma_list)

Définir le nombre d'étapes de diffusion $T$, les paramètres de diffusion $\beta_t$ et calculer les paramètres $\alpha_t$ :

In [ ]:
T = 50 #nombre de pas de diffusion
beta_list_temp = np.linspace(1e-2,2e-1,T-1)
beta_list = np.zeros(T)
beta_list[1:] = beta_list_temp #ajout de beta_0 = 1 par convention
alpha_list = np.cumprod((1.-beta_list)) #alpha_0 = 1 par convention
print(alpha_list)

**Question : Les valeurs des $\alpha_t$ sont affichées dans la console. À partir de ces valeurs, pensez-vous qu'à l'issue de la 50ème étape de diffusion les échantillons obtenus ressemblent aux échantillons d'une gaussienne centrée réduite ? Pourquoi ? (indice : regarder la dernière valeur de `alpha_list`)**  

Confirmez visuellement votre réponse en affichant les distributions analytiques :

In [ ]:
fig_gen, axs_gen = plt.subplots(T//5,5)
for t in range(T):
    plot_pdata_diffusion(w_list, mu_list, sigma_list, alpha_list[t], ax = axs_gen[t//5,t%5])
    plot_gaussian(ax = axs_gen[t//5,t%5])

Appliquer la diffusion aux éléments de la base de données, et calculer les histogrammes pour vérifier que tout est cohérent :

In [ ]:
nbins = 100
#n, bins, patches = axs_gen[0,0].hist(X.numpy(), nbins, density=True, facecolor='g', alpha=0.75)
plt.pause(0.1)
Z_prec = X.clone()
for t in range(T):
    #Z_i = math.sqrt(1.-beta_list[i])*Z_prec + math.sqrt(beta_list[i])*t.normal(mean=0., std=1., size=(N_samp,1))
    Z_t = math.sqrt(alpha_list[t])*X + math.sqrt(1-alpha_list[t])*torch.normal(mean=0., std=1., size=(N_samp,1))
    Z_prec = Z_t.clone()
    n, bins, patches = axs_gen[t//5,t%5].hist(Z_t.numpy(), nbins, density=True, facecolor='g', alpha=0.75)

plt.pause(0.7)

# Réseau débruiteur
**Si besoin, commenter les affichages précédents pour accélérer l'éxecution du code.**  
  
Il faut désormais choisir une architecture de réseau de neurones $\mu_\theta (Z_t, t\rightarrow t-1)$ permettant de débruiter $Z_t$ depuis l'instant $t$ vers l'instant $t-1$, ce qui est équivalent (avec la paramétrisation de la moyenne vue en cours) à considérer un réseau prédisant le bruit ayant été ajouté $\hat\epsilon_\theta (Z_t, t\rightarrow t-1)$. Par convention, en cours, nous avons remplacé la notation $t\rightarrow t-1$ par le scalaire $t-1$. C'est ce scalaire qui est passé en entrée du réseau, et qui indique au réseau le traitement à effecteur (**il ne s'agit que d'une convention, on peut tout aussi bien choisir de remplacer $t\rightarrow t-1$ par le scalaire $t$ plutôt que $t-1$**).
  
**À coder : Implémenter un perceptron multicouche avec deux couches cachées (FC->tanh->FC->tanh->FC). Les deux entrées scalaires $Z_t$ et $t-1$ seront transformées en un vecteur de dimension 3 : $[Z_t, cos(\frac{t-1}{T}), sin(\frac{t-1}{T})]$.**

# Apprentissage des paramètres du réseau débruiteur


**À coder  : Implémenter la technique d'apprentissage du réseau débruiteur vue en cours, consistant à générer des minibatches, où pour chaque élément $X_i$ du minibatch, un temps de diffusion $t_i$ est tiré aléatoirement entre $1$ et $T$, conduisant à la fonction de coût suivante :**  
**$\sum_{i=1}^{\text{batch size}}(\epsilon_i - \hat\epsilon_\theta (\sqrt{\alpha_{t_i}}X_i+\sqrt{1-\alpha_{t_i}}\epsilon_i, t_i-1))^2$ avec $\epsilon_i\sim\mathcal{N}(0,1)$.**  

**Réaliser des affichages (coût, sortie du réseau à chaque pas de temps, génération de nouveaux échantillons,...) pour s'assurer que votre implémentation fonctionne correctement. Remarque : L'implémentation de ces affichages devrait prendre un certain du temps, en particulier la partie génération de nouveaux échantillons.**

Une correction vous est proposée ci-après.

In [ ]:
import math
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import sys

def plot_pdata_diffusion(w_list, mu_list, sigma_list, alpha_diff, ax=None, orientation='vertical'):
    
    x = torch.linspace(start=-6., end=8., steps=1000)
    
    p_data = torch.zeros_like(x)
    N_comp = w_list.shape[0]
    for i in range(N_comp):
        std_diff = math.sqrt(alpha_diff*(sigma_list[i]**2)+(1.-alpha_diff))
        p_data_i = (1./(std_diff*math.sqrt(2*math.pi))*torch.exp(-0.5*((x-np.sqrt(alpha_diff)*mu_list[i])/std_diff)**2))
        p_data += w_list[i]*p_data_i
        
    if(ax==None):
        if(orientation=='vertical'):
            plt.plot(x,p_data,'k')
        else:
            plt.plot(p_data,x,'k')
    else:
        if(orientation=='vertical'):
            ax.plot(x,p_data,'k')
        else:
            ax.plot(p_data,x,'k')
    return

def plot_gaussian(ax=None, orientation='vertical'):
    
    x = torch.linspace(start=-6., end=8., steps=1000)
        
    p_data = (1./(math.sqrt(2*math.pi))*torch.exp(-0.5*(x)**2))
        
    if(ax==None):
        if(orientation=='vertical'):
            plt.plot(x,p_data,'r-')
        else:
            plt.plot(p_data,x,'r-')
    else:
        if(orientation=='vertical'):
            ax.plot(x,p_data,'r-')
        else:
            ax.plot(p_data,x,'r-')
    return

def sample_from_pdata(N, w_list, mu_list, sigma_list):
    
    n_c = w_list.shape[0]

    samp = torch.zeros((N,1))
    mask = torch.multinomial(w_list,num_samples=N,replacement=True)
    
    for i in range(n_c):
        samp_i = torch.normal(mean=mu_list[i], std=sigma_list[i], size=(N,1))
        samp[mask==i] = samp_i[mask==i]
 
    return samp



w_list = torch.tensor([0.2, 0.45, 0.35, 0.1, 0.3])
w_list /=w_list.sum() 

mu_list = torch.tensor([-3., 2.5, 1.5, -2.5, 5.])
sigma_list = torch.tensor([0.3, 1.2, 0.3, 0.2, 0.1])

T = 50
beta_list_temp = np.linspace(1e-2,2e-1,T-1)
#beta_list_temp = np.array([5e-2, 5e-2, 5e-2, 5e-2, 1e-1, 5e-1, 5e-1, 5e-1, 8e-1])#np.linspace(1e-2,1e-1,T)#np.array([5e-2, 5e-2, 5e-2, 5e-2, 1e-1, 5e-1, 5e-1, 5e-1, 5e-1])#np.linspace(5e-3,2e-1,9)
beta_list = np.zeros(T)
beta_list[1:] = beta_list_temp #ajout de beta_0 = 1 par convention
print(beta_list)
alpha_list = np.cumprod((1.-beta_list)) #alpha_0 = 1 par convention
print(alpha_list)

N_samp = int(2e4)
X = sample_from_pdata(N_samp, w_list, mu_list, sigma_list)

#sys.exit()    
class denoiser(nn.Module):
    def __init__(self,H):
        super(denoiser, self).__init__()
        
        self.H = H
        
        self.linearIn = nn.Linear(3, H)
        self.activIn = nn.Tanh()
        
        self.linearHidden = nn.Linear(H, H)
        self.activHidden = nn.Tanh()
        
        self.linearOut = nn.Linear(H, 1)
 
    def forward(self, z, t):

        x = torch.cat((z, torch.cos(t), torch.sin(t)), dim=1)
        out = self.linearIn(x)
        out = self.activIn(out)
        
        out = self.linearHidden(out)
        out = self.activHidden(out)
        
        noise = self.linearOut(out)

        return noise
    
T = len(beta_list)
H = 300
learning_rate = 1e-3
batchSize = 2048#256

den = denoiser(H)
optimizer = torch.optim.Adam(den.parameters(), lr=learning_rate)    
NItMax = 5000
alpha_list_t = torch.tensor(alpha_list).float()

fig_map, axs_map = plt.subplots(T//5,5)
fig_est, axs_est = plt.subplots(T//5,5)
fig_curves, axs_curves = plt.subplots(1,1)
loss_v = np.nan*np.zeros(int(NItMax/100))
line_loss, = axs_curves.plot(np.linspace(0,NItMax,int(NItMax/100)),loss_v)


# fig = plt.figure(figsize=(12, 4))

for i in range(NItMax):

    if(i==4000):
        optimizer.param_groups[0]['lr'] /= 10.
    # if(i>10000):
    #     for g in optimizer.param_groups:
    #         g['lr'] = 1e-4

    
    perm = torch.randperm(N_samp)
    X_batch = X[perm[:batchSize],:].float()
    #t_batch = (T-1)*torch.ones((batchSize,1)).long()#
    t_batch = torch.randint(low=1, high=T, size=(batchSize,1))
    eps_batch = torch.normal(mean=0., std=1., size=(batchSize,1)).float()       
    Z_batch = torch.sqrt(alpha_list_t[t_batch])*X_batch + torch.sqrt(1-alpha_list_t[t_batch])*eps_batch
    t_batch_norm = (t_batch/T).float()
    
    eps_est = den(Z_batch,t_batch_norm)
    
    l = (((eps_est - eps_batch)**2).sum())/batchSize
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    
    print('It {} : loss : {:.2e}, lr : {}'.format(i, l.item(), optimizer.param_groups[0]['lr']))
    
    if(i%100 == 0):
        loss_v[int(i/100)] = l.data
        plt.figure(fig_curves.number)
        #line_loss.set_ydata(loss_v)
        #fig_curves.canvas.draw()
        axs_curves.clear()
        axs_curves.grid('on')
        line_loss, = axs_curves.plot(np.linspace(0,NItMax,int(NItMax/100)),loss_v)
        
        plt.pause(0.1)
    

    #if((i<1000 and i%200==0) or i%2000 == 0):
    if(i==NItMax-1):
        with torch.no_grad():
            
            plt.figure(fig_map.number)
            fig_map.suptitle('Iter {}'.format(i), fontsize=16)
            for t in reversed(range(1,T)):

                perm = torch.randperm(N_samp)
                X_batch = X[perm[:batchSize],:].float()
                t_batch = t*torch.ones((batchSize,1)).long()#
                eps_batch = torch.normal(mean=0., std=1., size=(batchSize,1)).float()       
                Z_batch = torch.sqrt(alpha_list_t[t_batch])*X_batch + torch.sqrt(1-alpha_list_t[t_batch])*eps_batch
                t_batch_norm = (t_batch/T).float()
                eps_est = den(Z_batch,t_batch_norm)
    
                Z_sort,ind = torch.sort(Z_batch[::10],dim=0)
                eps_batch_temp = eps_batch[::10]
                eps_batch_sort = eps_batch_temp[ind.view(-1)]
                eps_est_temp = eps_est[::10]
                eps_est_sort = eps_est_temp[ind.view(-1)]
                axs_map[(t-1)//5,(t-1)%5].clear()
                axs_map[(t-1)//5,(t-1)%5].plot(Z_sort, eps_batch_sort, label='a', color='b')
                axs_map[(t-1)//5,(t-1)%5].plot(Z_sort, eps_est_sort, label='b', color='r', linestyle='dashed')
                axs_map[(t-1)//5,(t-1)%5].grid('on')
                #axs_map[(t-1)//5,(t-1)%5].legend()
            
            plt.pause(0.7)

            plt.figure(fig_est.number)
            fig_est.suptitle('Iter {}'.format(i), fontsize=16)
            N_samp_val = 10000
            Z_prec = torch.normal(mean=0., std=1., size=(N_samp_val,1)).float()
            for t in reversed(range(1,T)):
                #t = T-1
                mu_t = (1./math.sqrt(1.-beta_list[t]))*(Z_prec - (beta_list[t]/math.sqrt(1.-alpha_list[t]))*den(Z_prec,(t/T)*torch.ones_like(Z_prec)))
                sigma_t = math.sqrt(beta_list[t])
                Z_t = mu_t + sigma_t*torch.normal(mean=0., std=1., size=(N_samp_val,1)).float()
                Z_prec = Z_t.clone()
                axs_est[(t-1)//5,(t-1)%5].clear()
                plot_gaussian(ax = axs_est[(t-1)//5,(t-1)%5])
                plot_pdata_diffusion(w_list, mu_list, sigma_list, alpha_list[t-1], ax = axs_est[(t-1)//5,(t-1)%5])
                nbins = 100
                n, bins, patches = axs_est[(t-1)//5,(t-1)%5].hist(Z_t.numpy(), nbins, density=True, facecolor='g', alpha=0.75)

            plt.pause(0.7)

# (Optionnel) MNIST

L'implémentation précedente concerne un exemple 1D. Modifier cette implémentation pour apprendre à générer de nouveaux échantillons de la base de données MNIST. Attention, une imagette de MNIST a 28x28 pixels, ainsi le temps d'apprentissage sera beaucoup plus long et nécessite une carte graphique (vous pouvez utiliser Google COLAB).